In [0]:
%pip install xarray netcdf4 boto3 botocore
%restart_python

In [0]:
import time
import glob
import os
start = time.time()
era5_path_list = [
    p for p in glob.glob('/Volumes/aer-processed/era5/daily_summary/*.nc')]
print(f'took {time.time()-start:.2f}s to read {len(era5_path_list)} files')


In [0]:
import os
import glob
import shutil
import time
import concurrent.futures

def copy_file(file_path):
    shutil.copy(file_path, target_dir)

max_workers = 20

target_dir = '/tmp/era5'
offset = 365
index = 0
os.makedirs(target_dir, exist_ok=True)
while index < len(era5_path_list):
    start = time.time()
    print(f'max workers: {max_workers} -- ', end='')
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(copy_file, era5_path_list[index:index+offset])
    print(f'Copy complete, took {time.time()-start:.2f}s')
    index += offset



In [0]:
dbutils.fs.ls('dbfs:/tmp/')